In [1]:
%matplotlib inline

In [2]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import PIL
import random
from IPython.core.display import Image, display

import matplotlib.pyplot as plt

/home/s.shimovolos/anaconda3/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [3]:
DEVICE_ID = 3
DEVICE = torch.device('cuda:%d' % DEVICE_ID)
torch.cuda.set_device(DEVICE_ID)

In [4]:
torch.cuda.current_device()

3

In [5]:
torch.cuda.get_device_name(DEVICE_ID)

'GeForce GTX 1080 Ti'

In [6]:
np.random.seed(100500)

def data2image(data):
    res = np.transpose(np.reshape(data ,(3, 32,32)), (1,2,0))
    return PIL.Image.fromarray(np.uint8(res))

def imshow(img):
    if isinstance(img, torch.Tensor): img = img.numpy().astype('uint8')
    plt.imshow(np.transpose(img, (1, 2, 0)))
    
def prediction2classes(output_var):
    _, predicted = torch.max(output_var.data, 1)
    predicted.squeeze_()
    classes = predicted.tolist()
    return classes

def make_solution_pytorch(net, input_tensor, a_batch_size):
    res = []
    net = net.eval()
    cur_pos = 0
    while cur_pos <= len(input_tensor):
        outputs = net(input_tensor[cur_pos:cur_pos+a_batch_size])
        res += prediction2classes(outputs)
        cur_pos += a_batch_size
    return res

In [7]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

class CifarDataset(Dataset):
    def __init__(self, X, Y=None, is_train=True, transform=None):

        self.X = X
        self.Y = Y
        
        if is_train: 
            self.Y = [item[0] for item in self.Y]
        else: 
            self.Y = None
        
        self.X = self.X.reshape((self.X.shape[0], 3, 32, 32))
        self.X = self.X.transpose((0, 2, 3, 1)) #приводим к виду (N, H, W, C)
        self.X = [Image.fromarray(img) for img in self.X]
                
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        
        sample = self.X[idx]

        if self.transform: sample = self.transform(sample)

        if self.Y is None: return sample
        else: return (sample, self.Y[idx])

In [8]:
#Надо поменять пути на свои до файлов с kaggle
DATA_PATH  = '/home/s.shimovolos/cifar100/cifar-100-python/'
train_path = 'train'
test_path  = 'homework_4_no_classes_test.npy'

import pickle

with open(DATA_PATH + train_path, 'rb') as fo:
    data = pickle.load(fo, encoding='bytes')

X_full, Y_full, = data[b'data'], data[b'fine_labels']
Y_full = np.array(Y_full).reshape(50000, 1)

data = np.load('/home/s.shimovolos/' + test_path)
X_test = data

In [9]:
print(X_test.shape)

(10000, 3072)


In [10]:
np_mean = np.mean([item[0].numpy() for item in CifarDataset(X_full, Y_full, transform=transforms.ToTensor())], axis=(0,2,3))
np_std = np.std([item[0].numpy() for item in CifarDataset(X_full, Y_full, transform=transforms.ToTensor())], axis=(0,2,3))

In [11]:

cifar_transform_norm = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

cifar_test_transform_norm = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize(torch.FloatTensor(np_mean), torch.FloatTensor(np_std))
]
)

In [12]:

dataset_train_norm = CifarDataset(X_full, Y_full, transform=cifar_transform_norm)
dataloader_train_norm = DataLoader(dataset_train_norm, batch_size=128,
                        shuffle=True, num_workers=4)

dataset_test_norm = CifarDataset(X_test, is_train=False, transform=cifar_test_transform_norm)
dataloader_test_norm = DataLoader(dataset_test_norm, batch_size=128,
                        shuffle=False, num_workers=1)

def train_network(a_net, 
                  a_device,
                  dataloader_train_norm=dataloader_train_norm,
                  a_epochs=164,
                  a_batch_size=64,
                  a_lr=0.1):
    
    train_acc = []
    net = a_net.to(a_device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(a_net.parameters(), lr=a_lr)

    
    for epoch in range(a_epochs):  # loop over the dataset multiple times
        
        print("Epoch ", epoch)
        
        net = net.train()        
        epoch_accuracy = 0.0
        epoch_iters = 0
        for item in dataloader_train_norm:
            
            epoch_iters += 1

            inputs = item[0].to(a_device)
            labels = item[1].long().to(a_device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_accuracy += accuracy_score(labels, prediction2classes(outputs))

        epoch_accuracy /= epoch_iters
        train_acc.append(epoch_accuracy)
        
        print(round(train_acc[-1], 4))

    print('Finished Training')
    
    plt.plot(train_acc, label='Train')
    plt.legend()
    plt.grid()

In [13]:
def conv3x3(a_in_planes, a_out_planes, a_stride=1):
    """
    Основной строительный блок конволюций для ResNet
    Включает в себя padding=1 - чтобы размерность сохранялась после его применения
    """
    return nn.Conv2d(a_in_planes, a_out_planes,  stride=a_stride,
                     kernel_size=3, padding=1, bias=False)

In [14]:
class CifarResidualBlock(nn.Module):
    def __init__(self, in_c, out_c, stride, dropRate=0.0):
        super(CifarResidualBlock, self).__init__()
        self.dropRate = dropRate
        self.in_c = in_c
        self.out_c = out_c
        self.stride = stride
        
        self.conv1 = conv3x3(in_c, out_c, stride)
        self.bn1 = nn.BatchNorm2d(in_c)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        
        self.dropout = nn.Dropout(dropRate)        
        self.conv2 = conv3x3(out_c, out_c)
        self.bn2 = nn.BatchNorm2d(out_c)
        
        if stride != 1 or in_c != out_c:
            self.shortcut = nn.Conv2d(in_c, out_c, kernel_size=1, stride=stride, bias=False)
        else :
            self.shortcut = None
            
    def forward(self, x):
        result = self.bn1(x)
        result = self.relu1(result)
        result = self.conv1(result)
        
        result = self.dropout(result)
        result = self.bn2(result)
        result = self.relu2(result)
        result = self.conv2(result)  
        
        if self.shortcut is not None:
            addition = self.shortcut(x)
        else :
            addition = x
        
        result += addition
        return result

class WideBlock(nn.Module):
    def __init__(self, num_layers, in_c, out_c, stride, dropRate=0.0):
        super(WideBlock, self).__init__()
        self.layer = self.make_layer(in_c, out_c, num_layers, stride, dropRate)
        
    def make_layer(self, in_c, out_c, num_layers, stride, dropRate):
        layers = []
        layers.append(CifarResidualBlock(in_c, out_c, stride, dropRate))
        for i in range(num_layers - 1):
            layers.append(CifarResidualBlock(out_c, out_c, 1, dropRate))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layer(x)

In [15]:
class WideResNet(nn.Module):
    def __init__(self, dropRate=0.0):
        super(WideResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16,  stride= 1,
                     kernel_size=5, padding=2, bias=False)
        
        self.block1 = WideBlock(4, 16, 160, 1, dropRate)   
        self.block2 = WideBlock(4, 160, 320, 2, dropRate)    
        self.block3 = WideBlock(4, 320, 640, 2, dropRate)
       
        self.bn1 = nn.BatchNorm2d(640)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(640, 100)
        self.nChannels = 640
        
        self.avgp = nn.AvgPool2d(8, 8)

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))

        out = self.avgp(out)
        out = out.view(-1, 640)
        out = self.fc(out)
        return out

In [16]:
resnet = WideResNet(dropRate=0.05)
%time train_network(resnet, torch.device(DEVICE), a_lr=0.005, a_epochs=25)
torch.save(resnet.state_dict(), "model")

Epoch  0
0.0695
Epoch  1
0.1575
Epoch  2


Process Process-12:
Process Process-11:
Process Process-10:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/s.shimovolos/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/s.shimovolos/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/s.shimovolos/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s.shimovolos/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/s.shimovolos/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/s.shimovolos/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, i

KeyboardInterrupt: 

In [17]:
def make_solution(a_net, a_device):
    res = []
    net = a_net.eval()
    for item in dataloader_test_norm:
        inputs = item.to(a_device)
        outputs = net(inputs) 

        res += prediction2classes(outputs)
    return res

In [18]:
my_solution = make_solution(resnet, DEVICE)

In [19]:
with open('my_solution.csv', 'w') as fout:
    print('Id', 'Prediction', sep=',', file=fout)
    for i, prediction in enumerate(my_solution):
        print(i, prediction, sep=',', file=fout)

In [20]:
epoch_iters = 0
epoch_accuracy = 0
for item in dataloader_train_norm:
            
    epoch_iters += 1

    inputs = item[0].cuda()
    labels = item[1].long().cuda()

    outputs = resnet(inputs)
    epoch_accuracy += accuracy_score(labels, prediction2classes(outputs))
epoch_accuracy /= epoch_iters
        
print("Acc ", epoch_accuracy)

Acc  0.918618126598
